##### Imports

In [ ]:
import sys
from pathlib import Path
import warnings
from torch.utils.data import Dataset

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import pickle
from pathlib import Path
from scr.dataset import *
# from scr.game import *
from scr.feature_engineering import *
# from scr.plot_utils import *
import gc
from scr.utils import print_scenarios


from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset

from scr.utils import read_words, save_words_to_file

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)

random.shuffle(word_list)

In [ ]:
# import h5py
# import json

# # Sample Hangman game data
# games_data = [
#     {
#         'guessed_states': ['___b____e', 'a_aba___e', 'anaba__ne', 'anaba_ine'],
#         'guessed_letters': ['a', 'n', 'i', 's'],
#         'word': 'anabasine',
#         'initial_state': '___b_____',
#         'attributes': {'game_state': 'early', 'difficulty': 'easy', 'outcome': 'win', 
#                           'word_length': 9}
#     },
#     {
#         'guessed_states': ['__c______', '_a_a_____'],
#         'guessed_letters': ['c', 'a'],
#         'word': 'cacophony',
#         'initial_state': '__c______',
#         'attributes': {'game_state': 'mid', 'difficulty': 'hard', 'outcome': 'lose', 
#                             'word_length': 9}
#     }
    
#     # ... more games
# ]

# # Creating the HDF5 file
# with h5py.File('HangmanData.h5', 'w') as f:
#     for i, game in enumerate(games_data):
#         # Create a group for each game
#         game_group = f.create_group(f'game_{i+1}')

#         # Serialize and store each part of the game data
#         for key, value in game.items():
#             if isinstance(value, dict):
#                 # If the value is a dictionary, serialize it to JSON
#                 value = json.dumps(value)
#             game_group.create_dataset(key, data=value)

##### Data reading and Params Settings

In [ ]:
#### Papermil if need
 
NUM_STRATIFIED_SAMPLES = 1_000 # This will be overwritten by Papermill

# Define the base directory
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")

# Create a subdirectory for the stratified samples
stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)
stratified_samples_dir.mkdir(parents=True, exist_ok=True)

print(stratified_samples_dir)

In [ ]:
# Splitting Dataset Function
import random

def split_dataset(word_list, train_ratio=0.8):
    total_words = len(word_list)
    train_size = int(total_words * train_ratio)
    random.shuffle(word_list)
    return word_list[:train_size], word_list[train_size:]

# Splitting the word list
train_words, test_words = split_dataset(word_list)

# Save split datasets to files
save_words_to_file(train_words, stratified_samples_dir / 'train_words.txt')
save_words_to_file(test_words, stratified_samples_dir / 'test_words.txt')

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(train_words)
char_frequency = calculate_char_frequencies(train_words)
max_word_length = max(len(word) for word in train_words)

In [ ]:
len(train_words)

##### Stratified Sample Generation

In [ ]:
## we are taking starified samples from train_words

from scr.custom_sampler import \
    stratified_sample_by_length_and_frequency, \
        stratified_sample_by_length, stratified_sample_by_length_and_uniqueness

print(f'Number of Statrified samples: {NUM_STRATIFIED_SAMPLES}')

# sampled_words_by_length_and_frequency \
#     = stratified_sample_by_length_and_frequency(train_words, \
#     word_frequencies, \
#     NUM_STRATIFIED_SAMPLES)

sampled_words_by_length = stratified_sample_by_length_and_uniqueness(word_list, \
    NUM_STRATIFIED_SAMPLES)


print(len(sampled_words_by_length))

##### Intial State Simulation Testing

In [ ]:
from scr.game import *

In [ ]:
# word = "mississippi"
word = "mythopoetize"
# word = "cat"

initial_states = process_word_for_six_states(word)

initial_states

In [ ]:
len(initial_states)

In [ ]:
initial_states

##### Dataset Generation: Simulation

In [ ]:
word = "mississippi"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States: ", initial_states)

In [ ]:
from scr.game import simulate_game_progress, \
    play_game_with_a_word, process_word

# Example word and initial state
# Example usage
word = "mississippi"
# word = "cat"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States:")
for state_name, initial_state in initial_states.items():
    # Simulate the game
    print(initial_state)
    print(f"For initial state: {initial_state}")
    won, game_progress = simulate_game_progress(
        model=None,  # Assuming model is not used in this example
        word=word, 
        initial_state=initial_state, 
        char_frequency={},  # Assuming char_frequency is not used in this example
        max_word_length=len(word), 
        device=None,  # Assuming device is not used in this example
        max_attempts=6, 
        normalize=True,
        difficulty="medium", 
        outcome_preference='win'
    )

    # Display game progress
    for step in game_progress:
        print(f"Guessed: '{step[0]}', New State: '{step[1]}', Correct: {step[2]}")

        # break

    # break

    # print("Game Result:", "Won" if won else "Lost")

##### Writing Parquet

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

# Assuming functions 'simulate_game_progress' and 'process_word_for_six_states' are defined elsewhere

def process_batch_to_parquet(batch, file_path, start_game_counter):
    game_counter = start_game_counter
    data_for_parquet = []

    for game_data in batch:
        word, state_name, initial_state, difficulty, outcome, won, game_progress = game_data
        if not game_progress:
            continue

        final_state = game_progress[-1][1]  # Get the final state from game_progress

        guessed_states = [initial_state] + [state for _, state, _ in game_progress]
        guessed_letters = [letter for letter, _, _ in game_progress]

        data_for_parquet.append({
            'game_id': game_counter,
            'word': word,
            'initial_state': initial_state,
            'final_state': final_state,
            'guessed_states': ','.join(guessed_states[:-1]),
            'guessed_letters': ','.join(guessed_letters),
            'game_state': state_name,
            'difficulty': difficulty,
            'outcome': outcome,
            'word_length': len(word),
            'won': won
        })

        game_counter += 1

    df = pd.DataFrame(data_for_parquet)
    table = pa.Table.from_pandas(df)
    pq.write_to_dataset(table, root_path=file_path, compression='snappy') #, existing_data_behavior='append')

    return game_counter

def generate_batch(start_index, end_index, sampled_words, game_states_func):
    batch = []
    for i in range(start_index, end_index):
        word = sampled_words[i]
        game_states = game_states_func(word)
        for state_name, initial_state in game_states.items():
            for difficulty in ["easy", "medium", "hard"]:
                for outcome in ["win", "lose"]:
                    won, game_progress = simulate_game_progress(
                        model=None, word=word, initial_state=initial_state,
                        char_frequency=char_frequency, max_word_length=max_word_length,
                        device=device, max_attempts=6, normalize=True,
                        difficulty=difficulty, outcome_preference=outcome
                    )
                    
                    batch.append((word, state_name, initial_state, difficulty, outcome, won, game_progress))
    return batch

def main_execution(sampled_words, game_states_func, batch_size, parquet_file_path):
    game_counter = 1
    total_words = len(sampled_words)

    print(f"Total number of words: ", total_words)

    # i = 0

    for start_index in tqdm(range(0, total_words, batch_size), desc="Processing Batches"):
        end_index = min(start_index + batch_size, total_words)
        batch = generate_batch(start_index, end_index, sampled_words, game_states_func)
        game_counter = process_batch_to_parquet(batch, parquet_file_path, game_counter)

        # if i < 5:
        #     i+=1
        #     break

    print(f"Total games processed: {game_counter - 1}")

# Execute the main function
# Main execution
batch_size = 1000  # Adjust based on your requirements

parquet_file_path = Path(stratified_samples_dir) / "parquets"
parquet_file_path.mkdir(exist_ok=True, parents=True)

main_execution(sampled_words_by_length, process_word_for_six_states, batch_size, parquet_file_path)

In [ ]:
parquet_file_path

In [ ]:
import pandas as pd

# # Replace this with the path to your Parquet file
# parquet_file_path = 'path/to/your/HangmanData.parquet'

# Read the Parquet file
df = pd.read_parquet(parquet_file_path)

# Display the first few rows of the DataFrame
df

In [ ]:
# Get the total number of rows (games) in the DataFrame
total_games = len(df)

print(f"Total number of games in the dataset: {total_games}")

# Additional checks you might want to perform:
# - Check for any null values or anomalies in the data
print(df.isnull().sum())

# - Get a summary of the DataFrame
print(df.describe())

# - Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"Number of unique words: {unique_words}")

# - Inspect the distribution of game outcomes, difficulties, etc.
print(df['outcome'].value_counts())
print(df['difficulty'].value_counts())

##### Reading Checking

In [ ]:
import os
import json
from pathlib import Path
from sklearn.model_selection import train_test_split

# # Assuming HangmanDataset class and other necessary variables are defined
# # parquet_file_path, NUM_STRATIFIED_SAMPLES, stratified_samples_dir

# def split_dataset(directory, test_size=0.20, random_state=42):
#     # Get the list of all parquet files
#     parquet_files = sorted([f for f in os.listdir(directory) if f.endswith('.parquet')])

#     # Split the list of files
#     train_files, valid_files = train_test_split(parquet_files, 
#                     test_size=test_size, random_state=random_state)

#     # Construct the full path for each file
#     train_files = [os.path.join(directory, f) for f in train_files]
#     valid_files = [os.path.join(directory, f) for f in valid_files]

#     return train_files, valid_files

import os
import pandas as pd
from sklearn.model_selection import train_test_split

def split_dataset(directory, test_size=0.20, random_state=42):
    train_dfs = []
    valid_dfs = []

    # Read each parquet file, perform train-test split, and store in lists
    for f in os.listdir(directory):
        if f.endswith('.parquet'):
            file_path = os.path.join(directory, f)
            df = pd.read_parquet(file_path)

            # Split the data from this file
            train_df, valid_df = train_test_split(df, test_size=test_size, random_state=random_state)
            train_dfs.append(train_df)
            valid_dfs.append(valid_df)

    # Concatenate all the train and validation splits
    full_train_data = pd.concat(train_dfs, ignore_index=True)
    full_valid_data = pd.concat(valid_dfs, ignore_index=True)

    return full_train_data, full_valid_data


# Split the dataset and obtain file paths
train_files, valid_files = split_dataset(parquet_file_path)

# Create the directory if it doesn't exist
Path(stratified_samples_dir).mkdir(parents=True, exist_ok=True)

# Define file paths for saving the dataset file paths
train_file_path = os.path.join(stratified_samples_dir, f'{NUM_STRATIFIED_SAMPLES}_train_files.json')
valid_file_path = os.path.join(stratified_samples_dir, f'{NUM_STRATIFIED_SAMPLES}_valid_files.json')

# Save the train and validation file paths
with open(train_file_path, 'w') as f:
    json.dump(train_files, f)
with open(valid_file_path, 'w') as f:
    json.dump(valid_files, f)


print(f"stratified_samples_dir: ", stratified_samples_dir)

# # Define file paths for saving the dataset file paths
# train_file_path = os.path.join(stratified_samples_dir, f'{NUM_STRATIFIED_SAMPLES}_train_files.json')
# valid_file_path = os.path.join(stratified_samples_dir, f'{NUM_STRATIFIED_SAMPLES}_valid_files.json')

# Load
with open(train_file_path, 'r') as f:
    train_files = json.load(f)

with open(valid_file_path, 'r') as f:
    valid_files = json.load(f)

# Recreate the datasets using the saved file paths
# Assuming HangmanDataset is your custom dataset class
train_dataset = HangmanDataset(train_files)
valid_dataset = HangmanDataset(valid_files)




In [ ]:
train_dataset[0]

In [ ]:
len(train_dataset)

In [ ]:
# Assuming hangman_dataset is already loaded and its total size is known
total_size = len(train_dataset)
last_element_index = total_size - 1

# Access the last element
last_data = train_dataset[last_element_index]

In [ ]:
last_data